<a href="https://colab.research.google.com/github/oli2tup/CE301/blob/main/Malaria_classification_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
from keras.layers.convolutional import Conv2D

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt

2.6.0


In [17]:
!git clone https://github.com/oli2tup/CE301.git


fatal: destination path 'CE301' already exists and is not an empty directory.


In [18]:
img_width = 100
img_height = 100

In [19]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2) # using 20% of data as validation

In [20]:
train_data_generator = datagen.flow_from_directory(
    directory ='/content/CE301/cell_images', target_size =(img_width, img_height),
    class_mode = 'binary',
    batch_size = 16,
    subset = 'training'
) # Training subset

Found 22048 images belonging to 2 classes.


In [21]:
validation_data_generator = datagen.flow_from_directory(
    directory ='/content/CE301/cell_images', target_size =(img_width, img_height),
    class_mode = 'binary',
    batch_size = 16,
    subset = 'validation'
) # Validation subset

Found 5510 images belonging to 2 classes.


In [15]:
train_data_generator.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [16]:
validation_data_generator.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

**MODEL**

In [25]:
model = Sequential()

model.add(Conv2D(16,(3,3), input_shape =(img_width, img_height, 3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)            

In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [31]:
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch = len(train_data_generator), 
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

TypeError: ignored

In [30]:
history.history

{'accuracy': [0.8282383680343628,
  0.9308327436447144,
  0.9360939860343933,
  0.9380442500114441,
  0.9442126154899597],
 'loss': [0.404231458902359,
  0.21712827682495117,
  0.1995220184326172,
  0.18585167825222015,
  0.1727733016014099],
 'val_accuracy': [0.9337568283081055,
  0.9355716705322266,
  0.9341197609901428,
  0.9359346628189087,
  0.9315789341926575],
 'val_loss': [0.21406103670597076,
  0.1863265037536621,
  0.21046490967273712,
  0.1790725439786911,
  0.19073472917079926]}

In [ ]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
      keras.layers.Conv2D(filters, 3, strides=strides,
        padding="same", use_bias=False),
      keras.layers.BatchNormalization(),
      self.activation,
      keras.layers.Conv2D(filters, 3, strides=1,
        padding="same", use_bias=False),
      keras.layers.BatchNormalization()]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
        keras.layers.Conv2D(filters, 1, strides=strides,
          padding="same", use_bias=False),
        keras.layers.BatchNormalization()]
  def call(self, inputs):
    Z  = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
      return self.activation(Z + skip_Z)